In [1]:
from utils.spark_utils import get_local_spark_session
from utils.spark_utils import cleanup_col_name, with_std_column_names

spark = get_local_spark_session()

22/02/25 18:38:59 WARN Utils: Your hostname, DESKTOP-9QGN9QQ resolves to a loopback address: 127.0.1.1; using 172.21.85.55 instead (on interface eth0)
22/02/25 18:38:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/02/25 18:39:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
spark

**Read postcode**

In [130]:
from pyspark.sql import functions as f
from utils.spark_utils import cleanup_col_name, with_std_column_names

postcode_df = spark.read.format('csv').option("header", "true").load("/mnt/c/users/lohith/Downloads/National_Statistics_Postcode_Lookup_UK_Coordinates.csv")
# postcode_df.printSchema()

postcode_df = postcode_df\
.filter("latitude IS NOT NULL OR longitude IS NOT NULL")\
.transform(with_std_column_names())\
.withColumn("postcode_geo",f.expr(
    """ ST_Transform(ST_Point(CAST(latitude AS DOUBLE), CAST(longitude AS DOUBLE)), 'epsg:4326', 'epsg:3857')""")
           )\
.selectExpr(
    "postcode_3 as postcode",
    "CAST(latitude AS DOUBLE) as lat",
    "CAST(longitude AS DOUBLE) as lon",
    "postcode_geo"
)

postcode_df.printSchema()
# postcode_df.createOrReplaceTempView("postcode_table")

root
 |-- postcode: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- postcode_geo: geometry (nullable = false)



**Read housePrices data**

In [123]:
from pyspark.sql.types import StructType, StructField, StringType
from typing import List

monthly_file_schema = StructType([
StructField('id',StringType(),True),
StructField('price',StringType(),True),
StructField('dt',StringType(),True),
StructField('postcode',StringType(),True),
StructField('type',StringType(),True),
StructField('new_flag',StringType(),True),
StructField('duration',StringType(),True),
StructField('poan',StringType(),True),
StructField('soan',StringType(),True),
StructField('street',StringType(),True),
StructField('locality',StringType(),True),
StructField('town',StringType(),True),
StructField('district',StringType(),True),
StructField('county',StringType(),True),
StructField('price_paid',StringType(),True),
StructField('record_status',StringType(),True),
])
hp_raw = spark.read.format('csv')\
.option("header", "false")\
.schema(monthly_file_schema)\
.load(f"/mnt/c/users/lohith/Downloads/housePriceRaw/")

hp = hp_raw.transform(with_std_column_names())\
.selectExpr(
    "postcode",
    "CAST(dt AS DATE) dt", 
    "type", 
    "poan",
    "CAST(price AS INT) as price", 
    "street",
    "town",
    "district",
    "county"
)


In [1]:
hp.sort("dt","type")

NameError: name 'hp' is not defined


**Write HousePrices with geom column**

In [215]:
hp\
.filter("postcode IS NOT NULL")\
.join(postcode_df,"postcode",'inner')\
.withColumn("postcode",f.expr("regexp_replace(postcode, ' ', '')"))\
.sort("lat", "lon")\
.repartition(4,'postcode_geo')\
.write.mode("overwrite")\
.save("/mnt/c/users/lohith/Downloads/house_price_lkp/")


**Prepare for the app run**

In [216]:
house_price_lkp = spark.read.load("/mnt/c/users/lohith/Downloads/house_price_lkp/")
post_code='BR6 6DN'
pdf = house_price_lkp\
.selectExpr('postcode','lat', 'lon').distinct().toPandas()

pdf_index = pdf.set_index('postcode')

In [224]:
house_price_lkp.unpersist()

DataFrame[postcode: string, dt: date, type: string, poan: string, price: double, street: string, town: string, district: string, county: string, lat: double, lon: double, postcode_geo: udt]

In [229]:
house_price_lkp = house_price_lkp.filter("year(dt)='2020'").cache()
house_price_lkp.take(1)
house_price_lkp.createOrReplaceTempView("housePriceLkp")


22/02/26 00:44:49 WARN CacheManager: Asked to cache already cached data.


In [285]:
# (lkp_lat,lkp_lon) = (59.37707668858785, -1.4762877837637978)
(lkp_lat,lkp_lon) = (51.574968, 0.088584)
# (lkp_lat,lkp_lon) = (53.80338074459603, -1.5591111308414478)


def get_house_prices(lkp_lat, lkp_lon):
    df = spark.sql(f"""
    SELECT 
    lat,
    lon,
    postcode,
    dt, 
    type, 
    price, 
    street,
    town,
    district,
    county,
    ST_Distance(
        postcode_geo,
        ST_Transform(ST_Point({lkp_lat},{lkp_lon}),'epsg:4326', 'epsg:3857')
        )/1609 AS distance_in_km
    FROM 
    housePriceLkp
    WHERE ST_Distance(
        postcode_geo,
        ST_Transform(ST_Point({lkp_lat},{lkp_lon}),'epsg:4326', 'epsg:3857')
        )/1609 < .5   
    """)
    df = df.withColumn("tooltip",f.expr(
        """CONCAT('£',price, '-', type, '-', dt) """)
                       )
    
    return df


# if get_house_prices(lkp_lat, lkp_lon).take(1):
#     print(1)

# get_house_prices(lkp_lat, lkp_lon).show(100)


In [267]:
# house_point

In [5]:
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc, html
import dash_leaflet.express as dlx
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import dash_leaflet as dl
from dash.exceptions import PreventUpdate

In [286]:

app = JupyterDash(__name__, 
                prevent_initial_callbacks=True,
                suppress_callback_exceptions=True,)

icons ={ 
    "iconUrl": 'https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-2x-yellow.png',
    "shadowUrl": 'https://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.7/images/marker-shadow.png',
    "iconSize": [25, 41],
    "iconAnchor": [12, 41],
    "popupAnchor": [1, -34],
    "shadowSize": [41, 41]
    }

app.layout = dbc.Container([
    
    dbc.Row(
        dbc.Col([
            html.Br(),
            html.Br(),
            postCodeIp := dcc.Input(placeholder="Input your postcode here"),
            submitButton := dbc.Button("Submit"),
            html.Br(),
            alertDummyContainer := html.Div(),
            alertDummyContainer1 := html.Div(),
            dcc.Store(id="inputLatLonStore")
        ])
    ),
    dbc.Row(
        dbc.Col([
            myMap := dl.Map([
                    dl.TileLayer(),
                    dl.LayerGroup(id="layer"),
                    dl.LayerGroup(id="housePriceLayer"),
                    dl.LayerGroup(id="housePriceLayerOnMapClick"),
                    # dl.LocateControl(options={'locateOptions': {'enableHighAccuracy': True}}),
                ], id="map",
                center=(51.354221748320946, 0.09599384789215022),
                zoom=15,
                style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"}),
        ])
    ),
], fluid=True)


@app.callback(
    Output("housePriceLayerOnMapClick", "children"),
    [Input("map", "click_lat_lng")],
    prevent_initial_call=True
)
def get_nearby_houses_on_mapclick(click_lat_lng):
    (lkp_lat,lkp_lon) = click_lat_lng[0], click_lat_lng[1]
    print(lkp_lat,lkp_lon)
    house_price_sdf = get_house_prices(lkp_lat,lkp_lon)
    
    house_price_pdf = house_price_sdf.toPandas()
    house_point = house_price_pdf.to_dict('records')
        
    if len(house_point)==0:
        return []
    
    house_price_on_map_click = dl.GeoJSON(
        id="someId",
        data=dlx.dicts_to_geojson(
            house_point,
        ),
        cluster=True,
        )
    
    return house_price_on_map_click

@app.callback(
    Output("housePriceLayer", "children"),
    Input("inputLatLonStore","data"),
    prevent_initial_call=True
)
def get_nearby_houses_on_postcode(latlon):
    if len(latlon)==0:
        raise PreventUpdate
        return []
        
    (lkp_lat,lkp_lon) = latlon[0].values()
    house_price_sdf = get_house_prices(lkp_lat,lkp_lon)
    
    house_price_pdf = house_price_sdf.toPandas()
    house_point = house_price_pdf.to_dict('records')
        
    if len(house_point)==0:
        return []
    
    house_price_layer = dl.GeoJSON(
        id="someId",
        data=dlx.dicts_to_geojson(
            house_point,
        ),
        cluster=True,
        )
    
    return house_price_layer

@app.callback(
    Output(alertDummyContainer, "children"),
    Output("layer", "children"),
    Output(myMap, "center"),
    Output("inputLatLonStore", "data"),
    Input(submitButton, "n_clicks"),
    State(postCodeIp, "value"),
    prevent_initial_call=True
)
def displayPostCodeOnMap(n_clicks, post_code):
    post_code = post_code.replace(" ", "").upper()
    
    try:
        (lat,lon) = pdf_index.loc[post_code][['lat','lon']]
    except KeyError:
            print("Post code not found")
            alert =  dbc.Alert("Post Code NOT FOUND !!!", id="alert-auto", is_open=True, duration=3000),
            return alert, [], (51.354221748320946, 0.09599384789215022), []
  
    marker = dl.Marker(
        position=[lat, lon], children=dl.Tooltip(f"{post_code}"), icon=icons
    )
    return [], [marker], (lat, lon), [{'lat':lat,'lon':lon}]

app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/


51.38379531261044 0.06820827323898106


51.38025564916925 0.09792020160503068


In [15]:
import dash_leaflet as dl
import dash_leaflet.express as dlx
from dash import Dash, html
from dash_extensions.javascript import assign
from utils.params import HOST


# A few countries.
countries = [dict(name="Denmark", iso2="dk", lat=56.26392, lon=9.501785),
             dict(name="Sweden", iso2="se", lat=59.334591, lon=18.063240),
             dict(name="Norway", iso2="no", lat=59.911491, lon=9.501785)]
# Generate geojson with a marker for each country and name as tooltip.
geojson = dlx.dicts_to_geojson([{**c, **dict(tooltip=c['name'])} for c in countries])
# Create javascript function that draws a marker with a custom icon, in this case a flag hosted by flagcdn.
draw_flag = assign("""function(feature, latlng){
const flag = L.icon({iconUrl: 'https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-2x-yellow.png', iconSize: [64, 48]});
return L.marker(latlng, {icon: flag});
}""")

# draw_flag = assign("""function(feature, latlng){
# const flag = L.icon({iconUrl: `https://flagcdn.com/64x48/${feature.properties.iso2}.png`, iconSize: [64, 48]});
# return L.marker(latlng, {icon: flag});
# }""")
# Create example app.
app = JupyterDash()
app.layout = html.Div([
    dl.Map(children=[
        dl.TileLayer(), dl.GeoJSON(data=geojson, options=dict(pointToLayer=draw_flag), zoomToBounds=True)
    ], style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"}, id="map"),
])


app.run_server(mode="external")

Dash app running on http://127.0.0.1:8050/
